## Assignment 9 - Monica Zhang
https://spark.apache.org/docs/latest/ml-features#lsh-operations  
https://databricks.com/blog/2017/05/09/detecting-abuse-scale-locality-sensitive-hashing-uber-engineering.html

In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

#### Add "eagerEval.enabled" to beautify the way Spark DF is displayed

In [4]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [5]:
df = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/monicazhang/final_eda_2.parquet')
df.count()

23/03/06 20:29:49 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


626090

EDA

In [6]:
df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries,hashtags,tweet_text_clean
null,Thu Jul 07 06:26:...,null,"{[], null, [], []...",null,null,0,false,low,null,1544930737370959872,1544930737370959872,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,CharlieEatonPhD,"{null, Thu Jul 07...","<a href=""http://t...",RT @CharlieEatonP...,1657175165871,false,Let them eat stud...,"{false, Tue Nov 1...",null,[],let them eat stud...
null,Mon Jun 27 07:09:...,"[101, 140]","{[], null, [], [{...",null,"{[101, 372], {[],...",0,false,low,null,1541317826190790656,1541317826190790656,80sbestdecade_,1541301603134750720,1541301603134750720,1387387181493325828,1387387181493325828,false,en,null,null,0,null,null,null,null,null,0,0,,_Ephox_ @sgsgsg42...,null,"<a href=""https://...",@80sbestdecade_ @...,1656313780721,true,@80sbestdecade_ @...,"{false, Sun Oct 0...",null,[],80sbestdecade eph...
null,Wed Aug 24 17:36:...,null,"{[], null, [], []...",null,null,0,false,low,null,1562494038468485120,1562494038468485120,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,MsBlaireWhite,"{null, Wed Aug 24...","<a href=""http://t...",RT @MsBlaireWhite...,1661362583163,false,“College students...,"{false, Mon Jul 2...",null,[],college students ...
null,Wed Aug 24 17:36:...,null,"{[], null, [], []...",null,null,0,false,low,null,1562494049223065600,1562494049223065600,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,BerniceKing,"{null, Wed Aug 24...","<a href=""http://t...",RT @BerniceKing: ...,1661362585727,false,Many approve of l...,"{false, Sun Nov 0...",null,[],many approve of l...
null,Wed Aug 24 17:36:...,null,"{[], null, [], []...",null,null,0,false,low,null,1562494053358325763,1562494053358325763,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,larryelder,"{null, Tue Aug 23...","<a href=""http://t...",RT @larryelder: “...,1661362586713,false,“The availability...,"{false, Sat Oct 2...",null,[],the availability ...


In [17]:
keywords = ("student loan", "student loans", "student debt", "student debts",\
            "loan forgiveness", "student debt forgiveness", "student loan relief",\
            "college debt", "higher education debt", "financial aid", "debt-free college", "debt-free education",\
            "free college", "student loan forgiveness",\
            "studentloan", "studentloans", "studentdebt", "studentdebts", "loanforgiveness",\
            "studentdebtforgiveness", "studentloanrelief", "cancelstudentloans", 'cancelstudentdebt'\
            'cancelstudentdebtnow','nostudentdebt', "takeactioncanceldebt", 'freecollege', 'studentloanforgiveness'\
           'federalstudentloans', 'studentloandebt', 'cancelstudentloandebt') 

In [32]:
df.write.parquet('gs://msca-bdp-students-bucket/shared_data/monicazhang/final_eda_2.parquet')

In [ ]:
#df = df.withColumn("user_name", lower(df["user.name"]))

In [ ]:
#df = df.withColumn("user_description", lower(df["user.description"]))

In [ ]:
#df = df.withColumn("user.url", lower(df["user.url"]))

In [ ]:
df.groupBy('user.verified').count()

Define user type based on user description, user name, and if user is verified.

In [ ]:
df = df.withColumn(
    "user_type",
    when(
        (col('user.verified')=='true') & (df.user.url.contains("gov")| df.user_description.contains("government")), "government"
    ).when(
        (col('user.verified')=='true') & (df.user.url.contains("edu")| col('user_name').contains('school') | \
        col('user_name').contains('university') | df.user_description.contains("university")|\
        df.user_description.contains("school")), "school or university"
    ).when(
        (col('user.verified')=='true') & df.user.url.contains("org")|df.user_description.contains("nonprofit")| \
        df.user_description.contains("non-profit"), "nonprofit" #or non-profit
    ).when(
        (col('user.verified')=='true') & (col('user_name').contains("times")| col('user_name').contains('journal')|\
                                        col('user_name').contains('post')|col('user_name').contains('news') |\
                                          col('user_name').contains('news') | col('user_name').contains('chronicle')|\
                                          col('user_name').contains('today') | df.user_description.contains("news")), "news"
    ).when(
        (col('user.verified')=='true') & (df.user.followers_count>=100000), "influencer"
    ).otherwise("other"))

In [ ]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Sat, 04 March 2023 18:30:02'